<a href="https://colab.research.google.com/github/Anjan-AI/TSAI-EVA5/blob/master/EVA5_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing all the Python Packages & torch Library.
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #Input- 1X28X28 | Output- 32X28X28 |RF - 3X3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #Input- 32X28X28 | Output- 64X28X28 |RF - 5X5
        self.pool1 = nn.MaxPool2d(2, 2)              #Input- 64X28X28 | Output- 64X14X14 |RF - 10X10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #Input- 64X14X14 | Output- 128X14X14 |RF - 12X12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #Input- 128X14X14 | Output- 256X14X14 |RF - 14X14
        self.pool2 = nn.MaxPool2d(2, 2)                #Input- 256X14X14 | Output- 256X7X7 |RF - 28X28
        self.conv5 = nn.Conv2d(256, 512, 3)            #Input- 256X7X7 | Output- 512X5X5 |RF - 30X30
        self.conv6 = nn.Conv2d(512, 1024, 3)            #Input- 512X5X5 | Output- 1024X3X3 |RF - 32X32
        self.conv7 = nn.Conv2d(1024, 10, 3)             #Input- 512X5X5 | Output- 10X1X1 |RF - 34X34
        #self.conv8 = nn.Conv2d(10, 10, 3, padding=1)   # self added code

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # conv1->relu->conv2->relu->pooling [ Input : 1X28 X28 , Output : 64X14X14]
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # conv3->x relu->conv4->pooling2 [ Input 64 X14 X14 , output :256X7X7]
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
       # x = F.relu(self.conv7(x)) # removing relu form the last layer makes the accuracy to 98%. as it sets all th -ve valus to 0 and den the system is unable to 
                                   #learn the -ve features 
        x = self.conv7(x)
      #  x = self.conv8(x)   ## self added code
        x = x.view(-1, 10)   # [ input : 10 X1X1 , output : 1X10]
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:

# seed the model to obtain consistent results
torch.manual_seed(1)
# this is the batch size , in 1 pas no of images passed together.
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# load the training data and perform standard normalization 
# parameter for normalization is mean and std dev.
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

#lad the test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [5]:
from tqdm import tqdm
# Function to train 
'''
Args: 
Model : created model to be used for training
device : GPU or cpu
train_laoded: data on which the training has to be done
Optimizer : the optimization algorithm to be used
epoch : no fo epoch 

'''
def train(model, device, train_loader, optimizer, epoch):
    model.train() # Set the model on training mode
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # moving the data to device
        optimizer.zero_grad() # zero the graidents 
        output = model(data)  # getting the model output
        loss = F.nll_loss(output, target) ## calculating the The negative log likelihood loss
        loss.backward() # flowing the gradients backward.
        optimizer.step() # paameter updated basd on the current gradient.
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

# Function to test 
'''
Args: 
Model : created model to be used for training
device : GPU or cpu
test_laoded: data on which the testing has to be done
 

'''

def test(model, device, test_loader):
    model.eval()  # seting up the model for evalaution.
    test_loss = 0  # setting the test loss to 0
    correct = 0    # countign the no of coorect classfication.
    with torch.no_grad():   # # turn off gradients, since we are in test mode
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # copy the data to device.
            output = model(data)  # predict the output
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset) # calculating hte test loss.

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device) # move the model to device.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # intiating he SGD optimizer

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.03553972393274307 batch_id=468: 100%|██████████| 469/469 [00:37<00:00, 12.56it/s]



Test set: Average loss: 0.0655, Accuracy: 9796/10000 (98%)

